# Counting Crowds with DL
## Proof Of Concept
The notebook will implement [Dense Scale Networks](https://arxiv.org/pdf/1906.09707.pdf) for the purpose of counting crowds for images.
The dataset will be the same as the one, detailed in the paper - the ShangaiTech.

## 1. Imports and loading the data.

In [1]:
import os
import pandas as pd
import numpy as np
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.metrics import mse, mae

Using TensorFlow backend.


In [20]:
import scipy.io

gt_file = scipy.io.loadmat('training_dataset/ShanghaiTech/part_A/train_data/ground-truth/GT_IMG_1.mat')

In [43]:
gt_file['image_info']

array([[array([[(array([[ 29.6225116 , 472.92022152],
       [ 54.35533603, 454.96602305],
       [ 51.79045053, 460.46220626],
       ...,
       [597.89732076, 688.27900015],
       [965.77518336, 638.44693908],
       [166.9965574 , 628.1873971 ]]), array([[1546]], dtype=uint16))]],
      dtype=[('location', 'O'), ('number', 'O')])]], dtype=object)

## 2. Building the Model
In order to build ithe Deep scale network first the Dense Dialated Convolution Block has to be implemented.

In [2]:
class ConvBlock(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, activation='relu', padding=1, dilation_rate=1, **kwargs):
        super.__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(256, (1, 1), activation=activation)
        self.padding = keras.layers.ZeroPadding2D(padding=(padding, padding))
        self.conv2 = keras.layers.Conv2D(64, (3, 3), padding='valid', dilation_rate=dilation_rate, activation=activation)
    def call(self, inputs):
        out = self.conv1(inputs)
        out = self.padding(out)
        out = self.conv2(out)
        
        return out    

In [3]:
class DDCB(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.convBlock1 = ConvBlock()
        self.convBlock2 = ConvBlock(padding=2, dilation_rate=2)
        self.convBlock3 = ConvBlock(padding=3, dilation_rate=3)
        self.paddingOut = keras.layers.ZeroPadding2D(padding=(1,1))
        self.convOut = keras.layers.Conv2D(512, (3, 3), padding='valid', activation='relu')
    
    def call(inputs):
        out1 = self.convBlock1(inputs)
        out2 = keras.layers.Concatenate([inputs, out1])
        out3 = self.convBlock2(out2)
        out4 = keras.layers.Concatenate([inputs, out1, out3])
        out5 = self.convBlock3(out4)
        out6 = keras.layers.Concatenate([inputs, out3, out5])
        out7 = self.paddingOut(out6)
        out8 = self.convOut(out7)
        return out8 

In [16]:
class DenseScaleNet(keras.models.Model):
    '''
        TODO: Add Docstring
    '''
    def __init__(self, model=None, input_shape=None):
        super().__init__(**kwargs)
        if model is None:
            if input_shape is None:
                raise Exception('A model could not have an input shape set to None.')
                
            model = self.__create_backbone(input_shape)
        
        self.model = model
        self.DDCB1 = DDCB()
        self.DDCB2 = DDCB()
        self.DDCB3 = DDCB()
        self.padding1 = keras.layers.ZeroPadding2D(padding=(1,1))
        self.conv1 = keras.layers.Conv2D(128, (3, 3), padding='valid', activation='relu')
        self.padding2 = keras.layers.ZeroPadding2D(padding=(1,1))
        self.conv2 = keras.layers.Conv2D(64, (3,3), padding='valid', activation='relu')
        self.outconv = keras.layers.Conv2D(1, (1, 1), activation='relu')
    
    def __create_backbone(self, input_shape):
        '''
            TODO: Add Doctstring
        '''
        vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet', input_shape=input_shape)
        model = keras.models.Sequential()
        
        # we want to copy the first ten layers from VGG16
        for layer in vgg16_model[:11]:
            model.add(layer)
        
        return model
        
    def call(self, inputs):
        out1 = self.model(inputs)
        out2 = self.DDCB1(out1)
        out3 = out2 + out3
        out4 = self.DDCB2(out3)
        out5 = out4 + out2 + out1
        out6 = self.DDCB3(out5)
        out7 = out6 + out4 + out2 + out1
        out = self.padding1(out)
        out = self.conv1(out)
        out = self.padding2(out)
        out = self.conv2(out)
        out = self.outconv(out)
        
        return out           